In [6]:
import numpy as np
import netCDF4 as nc
import sys
import os
os.chdir('..')

In [8]:
data = nc.Dataset('solar/data/kaggle_solar/train/dswrf_sfc_latlon_subset_19940101_20071231.nc')
data

<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format UNDEFINED):
    Conventions: CF-1.0
    title: Subset of data from 2nd-generation multi-decadal ensemble reforecast generated from the NCEP Global Ensemble Forecast System, mimicking version operational at NCEP/EMC circa mid-2012.
    institution: NOAA Earth System Research Laboratory (ESRL)
    source: NCEP GFS v 9.01, T254L42.  Control initial conditions from CFSRR.  Perturbed initial conditions from ETR.  Model error simulated with STTP.
    references: http://www.esrl.noaa.gov/psd/forecasts/reforecast2/index.html
    history: Subset created 2013-01-15 19:19:01 UTC
    comment: Original dataset generated on DOE's supercomputers at Lawrence Berkeley Laboratory through ALCC/ASCR grant.
    dimensions(sizes): time(5113), lat(9), lon(16), ens(11), fhour(5)
    variables(dimensions): float64 time(time), int32 intTime(time), float32 lat(lat), float32 lon(lon), int16 ens(ens), int32 fhour(fhour), int32 intValidTi

In [9]:
for v in data.variables.keys():
    print v

time
intTime
lat
lon
ens
fhour
intValidTime
Downward_Short-Wave_Rad_Flux


In [10]:
dswrf = data.variables.values()[-1]

In [11]:
dswrf

<type 'netCDF4._netCDF4.Variable'>
float32 Downward_Short-Wave_Rad_Flux(time, ens, fhour, lat, lon)
    _FillValue: 9999.0
    units: W m-2
    long_name: Downward_Short-Wave_Rad_Flux_Average (Average for  Mixed Intervals) @ surface
    cell_methods: time: mean
    GRIB_param_discipline: Meteorological_products
    GRIB_param_category: Short-wave_Radiation
    GRIB_param_name: Downward_short_wave_rad_flux
    GRIB_generating_process_type: Forecast
    GRIB_param_id: [  2   0   4 192]
    GRIB_product_definition_template: 8
    GRIB_product_definition_template_desc: Average, accumulation, extreme values or other statistically processed value at a horizontal level in a time interval
    GRIB_level_type: 1
    GRIB_level_type_name: surface
    GRIB_interval_stat_type: Average
    GRIB_VectorComponentFlag: easterlyNortherlyRelative
unlimited dimensions: 
current shape = (5113, 11, 5, 9, 16)
filling on

In [12]:
dswrf.shape

(5113, 11, 5, 9, 16)

In [13]:
dswrf.dimensions

(u'time', u'ens', u'fhour', u'lat', u'lon')

In [14]:
for d in data.dimensions.values():
    print d

<type 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 5113

<type 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 9

<type 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 16

<type 'netCDF4._netCDF4.Dimension'>: name = 'ens', size = 11

<type 'netCDF4._netCDF4.Dimension'>: name = 'fhour', size = 5



In [15]:
lat = data.variables['lat']
lat.shape

(9,)

In [16]:
l = lat[:]
l

array([ 31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.], dtype=float32)

In [17]:
np.where(l == 33)

(array([2]),)

In [18]:
np.where(l == 33.)[0][0]

2

In [19]:
lon = data.variables['lon']
n = lon[:]
n

array([ 254.,  255.,  256.,  257.,  258.,  259.,  260.,  261.,  262.,
        263.,  264.,  265.,  266.,  267.,  268.,  269.], dtype=float32)

In [20]:
np.where(n == 260.)[0][0]

6

So, to find the data for a point at (33,260)

In [21]:
p_33_260 = dswrf[:,:,:,np.where(l == 33)[0][0],np.where(n == 260)[0][0]]

In [22]:
p_33_260.shape

(5113, 11, 5)

In [23]:
p = np.mean(p_33_260, axis = 1)

In [24]:
p.shape

(5113, 5)

In [25]:
p[1000,2]

296.81818

In [26]:
p = p_33_260.reshape(p_33_260.shape[0], p_33_260.shape[1] * p_33_260.shape[2])

In [27]:
p.shape

(5113, 55)

In [28]:
np.savetxt('33_260.csv',p,delimiter=',',fmt="%.06f")

In [29]:
dswrf[0,0,0,0,0]

0.0

In [30]:
dswrf1D = np.reshape(dswrf,-1)

In [31]:
len(dswrf1D)

40494960

In [32]:
dswrf1D.shape

(40494960,)

In [33]:
values = data.variables.values()

In [34]:
models = values[4][:]

In [35]:
models = [str(val) for val in models]

In [36]:
times = values[5][:]
times

array([12, 15, 18, 21, 24], dtype=int32)

In [37]:
import datetime
datetime.datetime.strptime(str(values[1][0]),"%Y%m%d%H")

datetime.datetime(1994, 1, 1, 0, 0)

In [38]:
import datetime
values = data.variables.values()
models = [str(val) for val in values[4][:]]
times = [datetime.time(val%24) for val in values[5][:]]
dates = [datetime.datetime.strptime(str(val), "%Y%m%d%H") for val in values[1][:]]

In [39]:
import pandas as pd

In [40]:
import itertools
new_indices = list(itertools.product(dates, models, times, values[2][:],values[3][:]))

In [41]:
len(new_indices)

40494960

In [42]:
dswrf_df = pd.DataFrame(dswrf1D)

In [ ]:
dswrf_df.head()

In [43]:
import numpy as np

In [44]:
index_frame = pd.DataFrame(new_indices)

In [45]:
index_frame.head()

,0,1,2,3,4
0,1994-01-01,0,12:00:00,31,254
1,1994-01-01,0,12:00:00,31,255
2,1994-01-01,0,12:00:00,31,256
3,1994-01-01,0,12:00:00,31,257
4,1994-01-01,0,12:00:00,31,258


In [46]:
dswrf_df.columns=['total_precip']

In [47]:
dswrf_df.head()

,total_precip
0,0
1,0
2,0
3,0
4,0


In [48]:
index_frame.columns=['date','model','time','latitude','longitude']

In [49]:
index_frame.head()

,date,model,time,latitude,longitude
0,1994-01-01,0,12:00:00,31,254
1,1994-01-01,0,12:00:00,31,255
2,1994-01-01,0,12:00:00,31,256
3,1994-01-01,0,12:00:00,31,257
4,1994-01-01,0,12:00:00,31,258


In [52]:
all_info = pd.merge(index_frame,dswrf_df,left_index=True, right_index=True)

In [53]:
type(new_indices)

list

In [54]:
one_loc_one_day = all_info[(all_info['latitude'] == 31) & (all_info['longitude'] == 254) & (all_info['date'] == '2004-03-07')]

In [92]:
one_loc_one_day.set_index(['date','time','model','latitude','longitude',]).unstack(['time','model','latitude','longitude'])

total_precip                                                        \
time           12:00:00 15:00:00 18:00:00 21:00:00 00:00:00 12:00:00 15:00:00   
model                 0        0        0        0        0        1        1   
latitude             31       31       31       31       31       31       31   
longitude           254      254      254      254      254      254      254   
date                                                                            
2004-03-07            0       50      308      820      637        0       60   

                                        ...                                \
time       18:00:00 21:00:00 00:00:00   ...    12:00:00 15:00:00 18:00:00   
model             1        1        1   ...           9        9        9   
latitude         31       31       31   ...          31       31       31   
longitude       254      254      254   ...         254      254      254   
date                                    ...                                 
2004-03-07      319      860      692   ...           0       60      317   

                                                                           
time       21:00:00 00:00:00 12:00:00 15:00:00 18:00:00 21:00:00 00:00:00  
model             9        9       10       10       10       10       10  
latitude         31       31       31       31       31       31       31  
longitude       254      254      254      254      254      254      254  
date                                                                       
2004-03-07      840      664        0       50      293      790      620  

[1 rows x 55 columns]

In [72]:
one_loc_one_day.head()

,date,model,time,latitude,longitude,total_precip
29446560,2004-03-07,0,12:00:00,31,254,0
29446704,2004-03-07,0,15:00:00,31,254,50
29446848,2004-03-07,0,18:00:00,31,254,308
29446992,2004-03-07,0,21:00:00,31,254,820
29447136,2004-03-07,0,00:00:00,31,254,637


In [56]:
dswrf1D

array([   0.,    0.,    0., ...,  120.,  120.,  150.], dtype=float32)

In [57]:
type(np.asarray(new_indices))

KeyboardInterrupt: 

In [ ]:
arrayed_index = np.asarray(new_indices)

In [ ]:
stacked = np.hstack((arrayed_index, dswrf1D[:,None]))